## Contradictory, My Dear Watson

Can machines determine the relationships between sentences?

Given two sentences, there are three ways they could be related:
* one could entail the other
* one could contradict the other
* they could be unrelated

In [2]:
import numpy as np
import pandas as pd

from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1726951878.662891      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0921 20:51:18.671331815      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0921 20:51:18.671348099      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0921 20:51:18.671351792      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0921 20:51:18.671354683      13 

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


## Set up the TPU

In [3]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # This is TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
  strategy = tf.distribute.get_strategy() #For CPU and/or single GPU
  print(f'Number of replicas: {strategy.num_replicas_in_sync}')

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1726951896.511544      13 service.cc:145] XLA service 0x5bcfdf92d600 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726951896.511600      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1726951896.511605      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1726951896.511608      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1726951896.511611      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1726951896.511613      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1726951896.511616      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1726951896.511619      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1726951896.511621      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

## Load the Data

| Label | Meaning | 
| --- | --- | 
| 0 | entailment|
| 1 | neutral|
| 2 | contradiction|

In [4]:
train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
test = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

train.groupby('label').sample(2, random_state=1).style.hide(axis='index')

id,premise,hypothesis,lang_abv,language,label
14fa262750,"The almost midtown Massabielle quarter (faubourg de Massabielle), is sometimes described as the most picturesque in the city.",The Massabielle quarter is considered the most photogenic.,en,English,0
b20c96d26b,Wanaweza pia kuwa wazuri baada ya kufunzwa.,Wanapopata mafunzo wanaweza kuwa wazuri kabisa,sw,Swahili,0
07e55edb23,"добре, добре, не преминавайте през всичко възможно",Можете просто да ми кажете края на историята.,bg,Bulgarian,1
b2e5c736a2,.Bằng cách tiếp cận các học sinh không tiếp cận được thông qua trường học và các tổ chức cộng đồng khác.,Các sinh viên đạt sẽ mãi mãi biết ơn,vi,Vietnamese,1
3673d73972,"Since the mid 1990s, aggregate household wealth has swelled relative to disposable personal income, largely due to increases in the market value of households' existing assets (see figure 1.2).",Aggregate household wealth has plummeted since the 1990s as household assets have steadily decreased.,en,English,2
5e37605f1d,"Sit down, will you?"" Tuppence sat down on the chair facing him.",He told Tuppence to get out.,en,English,2


train.hypothesis.str.split()

## Prepare Data for Input

We'll use a pretrained BERT model from HuggingFace.

First, we'll download the tokenizer.

Tokenizers turn sequences of words into arrays of numbers.

In [10]:
model_name = 'bert-base-multilingual-cased'

# We imported BertTokenizer from transformers
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Let's look at an example tokenization

In [11]:
def encode_sentence(sentence):
  # This is using the BERT Tokenizer that we just downloaded to generate tokens
  # So the sentence "I love machine learning" becomes
  # ['I', 'love', 'machine', 'learning', '!']
  tokens = list(tokenizer.tokenize(sentence))

  # Adding the separator token
  tokens.append('[SEP]')

  # This is using the BERT Tokenizer to convert the tokens to unique integers
  # So  ['I', 'love', 'machine', 'learning', '!'] becomes 
  # [146, 16138, 21432, 26901, 106]
  return tokenizer.convert_tokens_to_ids(tokens)

encode_sentence('I love machine learning')

[146, 16138, 21432, 26901, 102]

In [12]:
encode_sentence('I LOVE MACHINE LEARNING')

[146, 52734, 71008, 108880, 93280, 84977, 52188, 52898, 34065, 102]

BERT requires three inputs:
* input word IDs (what you see above)
* input masks
* input type IDs

These allow the model to know that the premise and hypothesis are distinct sentences and to ignore any padding from the tokenizer.

A `[CLS]` token is used to denote the beginning of the inputs and a `[SEP]` token is used to separate the premise and hypothesis.

We also need to pad all of the inputs to be the same size.

You can read more about BERT inputs at HuggingFace.

Now we can encode all premise/hypothesis pairs for input into BERT.

In [17]:
train.hypothesis.add(train.premise).apply(encode_sentence).apply(len).describe()

count    12120.000000
mean        46.483993
std         23.291278
min          2.000000
25%         30.000000
50%         43.000000
75%         59.000000
max        257.000000
dtype: float64

In [18]:
max_len=75

def bert_encode(hypotheses, premises, tokenizer, max_len):

  num_examples = len(hypotheses)

  # Encode the input sentences and convert the results to tensors
  hypoth_tensors = tf.ragged.constant(
      [encode_sentence(s) for s in np.array(hypotheses)]
  )

  premise_tensors = tf.ragged.constant(
      [encode_sentence(s) for s in np.array(premises)]
  )

  # Create the appropriate number of start tokens and then encode them
  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])] * num_examples

  # Create the input by combining the start token, the hypothesis, and the
  # premise. (Keep in mind that the separator token was added by the 
  # encode_sentences function.)
  # Don't forget to set the axis because the default is a vertical stack.
  input_word_ids = tf.concat([cls, hypoth_tensors, premise_tensors], axis=1)

  # The input mask is all ones (pay attention to everything?)
  input_mask = tf.ones_like(input_word_ids).to_tensor(
          shape=[input_word_ids.shape[0], max_len])

  # The type IDs are all zeros? Why?
  type_cls=tf.zeros_like(cls)
  type_hypoth = tf.zeros_like(hypoth_tensors)
  type_premise = tf.ones_like(premise_tensors)
  input_type_ids = tf.concat([type_cls, type_hypoth, type_premise], axis=1)\
    .to_tensor(
          shape=[input_word_ids.shape[0], max_len])

  # Combine all inputs into a dictionary
  inputs = {
      'input_word_ids': input_word_ids.to_tensor(
          shape=[input_word_ids.shape[0], max_len]),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids
  }

  return inputs

bert_encode(
    train.head(2).hypothesis.values,
    train.head(2).premise.values,
    tokenizer,
    max_len
)

{'input_word_ids': <tf.Tensor: shape=(2, 75), dtype=int32, numpy=
 array([[  101, 10117, 23123, 14628, 10106, 10105, 63313, 10309, 14499,
         14229, 10169, 11762, 61565, 10106, 21133,   119,   102, 10111,
         11762, 61565, 10309, 14289, 10106, 29659, 12141, 10105, 63313,
         23123,   119,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101, 46184, 15647, 10301, 10472, 63505, 10114, 11424, 10135,
         11762, 17850,   119,   102, 13252, 10301, 17850, 10189, 11951,
           191, 34189, 10284, 10169, 10106, 18194, 15647, 10108, 13255,
         84459,   117, 10833, 12415,   119,   102,     0,     0,     0,
             0,     0,     0,     0,   

In [20]:
train_input = bert_encode(
    train.hypothesis.values,
    train.premise.values,
    tokenizer,
    max_len
)

test_input = bert_encode(
    test.hypothesis.values,
    test.premise.values,
    tokenizer,
    max_len
)

print('Input data prepared for modeling')

Input data prepared for modeling


In [21]:
# We can see the encoding of the first test sentence here like this
test_input['input_word_ids'][0]

<tf.Tensor: shape=(75,), dtype=int32, numpy=
array([  101, 11076, 16577, 10691,   787, 26649, 40634, 59360, 50717,
       21735, 18779, 47238,   117, 13244, 22887, 12710,   829, 32245,
       11722, 85408, 10691, 69883, 23172, 13378, 10916, 11689, 12427,
       13141,   788, 10673, 17571,   119,   102,   764, 28744,   752,
       11076, 16577,   752, 10748, 67499, 10961,   752,   834, 82397,
       17317, 11242,   752, 11076, 16161,   752, 11076, 16161,   752,
       11363, 13244, 22887, 76528,   829, 32245, 11722, 85408, 10691,
       86325, 77970, 10429, 10691, 12995, 11689,   764, 28744, 13378,
       13244, 29847,   774], dtype=int32)>

## Create the Model

In [22]:
def build_model():
  # Load the BERT model from TensorFlow Hub
  bert_encoder = TFBertModel.from_pretrained(model_name)

  # Create the input layers for the model
  input_word_ids = tf.keras.Input(shape=(max_len,),
                                  dtype=tf.int32,
                                  name='input_word_ids')
  
  input_mask = tf.keras.Input(shape=(max_len,),
                              dtype=tf.int32,
                              name='input_mask')
  
  input_type_ids = tf.keras.Input(shape=(max_len,),
                                  dtype=tf.int32,
                                  name='input_type_ids')
  
  # Encode the input sentences
  # This creates a tensor of shape=(None, max_len, 768)
  embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]

  # Specify the output
  # This creates a tensor of shape=(None, 3) (since we have three classes?)
  output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0, :])

  # Create the model given the inputs and outputs
  model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids],
                         outputs=output)
  
  # Compile the model based on accuracy metric
  model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

# Instantiate the model and print summary
with strategy.scope():
  model = build_model()
  model.summary()

I0000 00:00:1726952252.409548      13 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initializ

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 75)]                 0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 75)]                 0         []                            
                                                                                                  
 input_type_ids (InputLayer  [(None, 75)]                 0         []                            
 )                                                                                                
                                                                                              

## Train the Model

In [11]:
model.fit(train_input,
          train.label.values,
          validation_split=0.2,
          epochs=2,
          verbose=1,
          batch_size=64)

Epoch 1/2
152/152 [==============================] - 3097s 20s/step - loss: 1.0430 - accuracy: 0.4500 - val_loss: 0.9270 - val_accuracy: 0.5664
Epoch 2/2
152/152 [==============================] - 2929s 19s/step - loss: 0.8369 - accuracy: 0.6231 - val_loss: 0.8849 - val_accuracy: 0.5969


## Make Predictions on the Test Set

In [ ]:
class_probabilities = model.predict(test_input)

predictions = class_probabilities.argmax(axis=-1)

submission = pd.DataFrame({'id': test.id, 'prediction': predictions})

submission.to_csv(
    'submission.csv',
    index=False)